# Working with CAS table as ImageTable

In [ ]:
%matplotlib inline
from swat import * # needed to start a CAS server.
sess = CAS('snap001.unx.sas.com', 14931, nworkers=2)

### Load dl_api 
**Note:** This is a temporary way to load the package, once we determine how to ship it, we will build the pip install accordingly.

In [ ]:
import sys
sys.path.append('\\\\sashq\\root\\dept\\cas\\leliuz\\DL_APIs')

In [ ]:
from dl_api.images import ImageTable 

### Load the CAS table

In [ ]:
sess.addcaslib(name='dnfs', path='/dept/cas/xianhu', dataSource=dict(srcType="DNFS"),activeonadd=False)
tr_tbl = sess.table.loadTable(caslib='dnfs', path='imagenet2012_train_resize_256.sashdat')['casTable']
te_tbl = sess.table.loadTable(caslib='dnfs', path='imagenet2012_valid_resize_256.sashdat')['casTable']

In [ ]:
tr_tbl.columninfo()

### Get ImageTable from CAS table

In [ ]:
tr_img = ImageTable.from_table(tr_tbl, image_col='_image_', label_col='labels', path_col='_path_')
te_img = ImageTable.from_table(te_tbl, image_col='_image_', label_col='labels', path_col='_path_')

In [ ]:
tr_img.show(nimages=8,ncol=4)

In [ ]:
tr_img.label_freq

In [ ]:
tr_img.image_summary

### Load the pre-built ResNet50

In [ ]:
from dl_api.applications import ResNet50

In [ ]:
ResNet50?

In [ ]:
model = ResNet50(sess,model_name='ResNet50', batch_norm_first=True, n_classes=1000, n_channels=3, width=224, height=224, 
               scale=1, random_flip='H', random_crop='UNIQUE', offsets=tr_img.channel_means)

In [ ]:
model.summary()

In [ ]:
model.plot_network()

In [ ]:
model.fit(data=tr_img, mini_batch_size=1, max_epochs=3, lr=1E-4)

### Plot the training history

In [ ]:
model.training_history

In [ ]:
model.plot_training_history(fig_size=(10,4))

### Scoring

In [ ]:
model.predict(te_img)

In [ ]:
model.valid_conf_mat

In [ ]:
model.plot_predict_res(type='C',image_id=2)

In [ ]:
model.plot_predict_res(type='M',image_id=0)

### Save model to astore

In [ ]:
model.deploy(path='/dept/cas/leliuz/DATA/Demo', output_format='astore')

In [ ]:
model.deploy(path='/dept/cas/leliuz/DATA/Demo', output_format='table')

In [ ]:
sess.endsession()